# Movies Imformation - Retrieval  

## Librarys

In [1]:
import os 
import nltk
import os
import string
import numpy as np
import copy
import pandas as pd
import pickle
import re
import math

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from collections import Counter
from num2words import num2words
from nltk.stem import WordNetLemmatizer
# Lemmatize with POS Tag
from nltk.corpus import wordnet
from num2words import num2words
from sklearn.metrics.pairwise import linear_kernel

title = "stories"
alpha = 0.4

## Import DataSet

In [26]:
smd = pd.read_csv('../the-movies-dataset/movies_metadata_equal_ratings.csv')
smd['description'] = smd['description'].fillna('')
smd['title'] = smd['title'].fillna('')
N = len(smd)

## Preprocessing

### Lowercase

In [3]:
def convert_lower_case(data):
    return np.char.lower(data)

### Stopwords

In [4]:
# Stop words are the most commonly occurring words which don’t give any
# additional value to the document vector. in-fact removing these will
# increase computation and space efficiency. 

In [5]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [6]:
def remove_stop_words(data):
    new_text = ""
    stop_words = stopwords.words('english')
    words = word_tokenize(str(data))

    for word in words:
        if word not in stop_words and len(word) > 1:
            new_text = new_text + " " + word
        
    return new_text        

### Punctuation

In [7]:
# (Punctuation - dấu chấm câu)are the unnecessary symbols that are in
# our corpus documents, we should be little careful with what we are 
# doing with this. There might be few problems such as U.S — us 
# “United Stated” being converted to “us” after the preprocessing. 
# hyphen and should usually be dealt little carefully. but for this 
# problem statement we are just going to remove these

In [8]:
def remove_punctuation(data):
    symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
    for i in range(len(symbols)):
        data = np.char.replace(data, symbols[i], ' ')
        data = np.char.replace(data, "  ", " ")
    data = np.char.replace(data, ',', '')
    return data

In [9]:
def remove_apostrophe(data):
    return np.char.replace(data, "'", "")

### Steamming

In [10]:
def stemming(data):
    new_text = ""
    words = word_tokenize(str(data))
    stemmer = PorterStemmer()
    
    for word in words:
        new_text = new_text + " " + stemmer.stem(word)
    return new_text

In [11]:
data = "The striped bats are hanging on their feet for best"
stemming(data)

' the stripe bat are hang on their feet for best'

### Lemmatizer words

#### Get word pos 

In [12]:
s = "i play so well"
nltk.pos_tag(s.split())

[('i', 'JJ'), ('play', 'VBP'), ('so', 'RB'), ('well', 'RB')]

In [13]:
s = "i play game"
nltk.pos_tag(word_tokenize(s))

[('i', 'JJ'), ('play', 'VBP'), ('game', 'NN')]

In [14]:
def get_wordnet_pos(word):

    tag = nltk.pos_tag([word])[0][1][0].upper()
    
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

#### Lemmatizer words

In [15]:
def lemmatizer_words(data):
    new_text = ""
    lemmatizer = WordNetLemmatizer()
    words = word_tokenize(str(data))
   
    lemmatizer_text = [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in words]

    for word in lemmatizer_text:
        new_text = new_text + " " + word
    return  new_text

In [16]:
data = "The striped bats are hanging on their feet for best"
data
lemmatizer_words(data)

' The strip bat be hang on their foot for best'

### Convert Number 

In [17]:
def convert_numbers(data):
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        try:
            w = num2words(int(w))
        except:
            a = 0
        new_text = new_text + " " + w
    new_text = np.char.replace(new_text, "-", " ")
    return new_text

In [18]:
data = "123"
convert_numbers(data)

array(' one hundred and twenty three', dtype='<U29')

### Combine all Preprocess step

In [19]:
def preprocess(data):
    data = convert_lower_case(data)
    data = remove_punctuation(data) #remove comma seperately
    data = remove_stop_words(data)
    data = remove_apostrophe(data)
    data = convert_numbers(data)
    data = lemmatizer_words(data)
    data = stemming(data)
    
    data = remove_punctuation(data)
    data = convert_numbers(data)
    data = lemmatizer_words(data)
    data = stemming(data) #needed again as we need to stem the words
    data = remove_punctuation(data) #needed again as num2word is giving few hypens and commas fourty-one
    data = remove_stop_words(data) #needed again as num2word is giving stop words 101 - one hundred and one
    return data

### Word tokenize data 

In [20]:
smd['description']

0       Led by Woody, Andy's toys live happily in his ...
1       When siblings Judy and Peter discover an encha...
2       A family wedding reignites the ancient feud be...
3       Cheated on, mistreated and stepped on, the wom...
4       Just when George Banks has recovered from his ...
                              ...                        
9020    The new installment of the Sharknado franchise...
9021    A man must cope with the loss of his wife and ...
9022    Rustom Pavri, an honourable officer of the Ind...
9023    Village lad Sarman is drawn to big, bad Mohenj...
9024    The band stormed Europe in 1963, and, in 1964,...
Name: description, Length: 9025, dtype: object

In [21]:
processed_text = []
processed_title = []

for index, row in smd.iterrows():
    description = row['description']
    title = row['title']   
    
    processed_text.append(word_tokenize(str(preprocess(description))))
    processed_title.append(word_tokenize(str(preprocess(title))))


In [22]:
processed_text[:1]

[['lead',
  'woodi',
  'andi',
  'toy',
  'live',
  'happili',
  'room',
  'andi',
  'birthday',
  'bring',
  'buzz',
  'lightyear',
  'onto',
  'scene',
  'afraid',
  'lose',
  'place',
  'andi',
  'heart',
  'woodi',
  'plot',
  'buzz',
  'circumst',
  'separ',
  'buzz',
  'woodi',
  'owner',
  'duo',
  'eventu',
  'learn',
  'put',
  'asid',
  'differ']]

In [23]:
processed_title[:10]

[['toy', 'stori'],
 ['jumanji'],
 ['grumpier', 'old', 'men'],
 ['wait', 'exhal'],
 ['father', 'bride', 'part', 'ii'],
 ['heat'],
 ['sabrina'],
 ['tom', 'huck'],
 ['sudden', 'death'],
 ['goldeney']]

In [48]:
smd = smd.reset_index()
titles = smd['title']
indices = pd.Series(smd.index, index=smd['title'])

## Create Corpus Bag 

In [27]:
# Mục đích là để tạo ra một bộ từ điển liệt kê tất cả các các từ và danh sách các bộ phim có sự xuất hiện của nó 


DF = {}

for i in range(N):
    tokens = processed_text[i]
    for w in tokens:
        try:
            DF[w].add(i)
        except:
            DF[w] = {i} #Trừ khi trong DF[w] có tồn tại i
    
    tokens = processed_title[i]
    for w in tokens:
        try:
            DF[w].add(i)
        except:
            DF[w] = {i}


In [28]:
# tính tần xuất xuất hiện của từ đó trên toàn bộ danh sách bộ phim 
for i in DF:
    DF[i] = len(DF[i])

In [29]:
total_vocab_size = len(DF)
total_vocab_size

22336

In [30]:
total_vocab = [x for x in DF]
total_vocab[:10]

['lead',
 'woodi',
 'andi',
 'toy',
 'live',
 'happili',
 'room',
 'birthday',
 'bring',
 'buzz']

In [31]:
def doc_freq(word):
    c = 0
    try:
        c = DF[word]
    except:
        pass
    return c

## Calcualte DF-IDF

### Calcalate DF-IDF from description

In [32]:
doc  = 0
tf_idf = {}

for i in range(N):
    
    tokens = processed_text[i]
    
    # counting frequence the number of word in a document
    counter = Counter(tokens + processed_title[i])
    words_count = len(tokens + processed_title[i])
    
    for token in np.unique(tokens):
        tf = counter[token]/words_count
        df = doc_freq(token)
        idf = np.log((N)/(df + 1))
        
        tf_idf[doc, token] = tf*idf
    
    doc += 1
    
tf_idf[(0,"toy")]    

0.3321095381255287

### Calcalate DF-IDF from title

In [33]:
tf_idf_title = {}
doc = 0

for i in range(N):
    
    tokens = processed_title[i]
    counter = Counter(tokens + processed_text[i])
    words_count = len (tokens + processed_text[i])
    
    for token in np.unique(tokens):
        tf = counter[token]/words_count
        df = doc_freq(token)
        idf = np.log(N/(df + 1))
        
        tf_idf_title[doc, token] = tf*idf
    doc += 1 
    
tf_idf_title[(0,"toy")]

0.3321095381255287

### Merging the TF-IDF according to weights

In [34]:
#TF-IDF(document) = TF-IDF(body) * alpha + TF-IDF(title) * (1-alpha)

In [35]:
for i in tf_idf:
    tf_idf[i] *= alpha

for i in tf_idf_title:
    tf_idf_title[i] *= (1 - alpha)

In [36]:
# shitty code
# thực ra nó lặp trên tf_idf nếu thằng nào mà có mặt trên cả title và body thì if_idf của nó tính bằng body và
# title đều như nhau.
# nếu 1 thằng word nào đó mà không có mặt trên tf_idf_title thì tf_idf_title[i] không tồn tại => safe, nó sẽ có 
# giá trị bằng tf_idf trên body nhân với alpha

In [37]:
for i in tf_idf_title:
    if i in tf_idf: tf_idf[i] = tf_idf[i] + tf_idf_title[i]
    else : tf_idf[i] = tf_idf_title[i]

tf_idf[(0,"toy")]

0.3321095381255287

### Mathching score method 

#### Matching score with query string 

In [38]:
# ý tưởng của mathching score là tìm tất cả các ký tự giống nhau giữa từng đoạn văn bản và query rồi tính tổng
# giá trị của các ký tự đó (đối với mỗi đoạn văn bản một) rồi lập list xếp theo giá trị query_weights

def matching_score(query):
    preprocessed_query = preprocess(query)
    tokens = word_tokenize(str(preprocessed_query))
    
    print("Matching score")
    print("\n Query:", query)
    print("")
    print(tokens)
    
    query_weights = {}
    
    for key in tf_idf:
        if key[1] in tokens:
            try: 
                query_weights[key[0]] += tf_idf[key]
            except:
                query_weights[key[0]] = tf_idf[key]
    
    query_weights = sorted(query_weights.items(), key=lambda x: x[1], reverse =True)
    
    
    print("")
    
    
    for i in query_weights[:10]:
         print(smd.iloc[i[0]]['title']) 

In [39]:
matching_score("the mafia god father ")

Matching score

 Query: the mafia god father 

['mafia', 'god', 'father']

Jane Austen's Mafia!
Dear God
Oh, God!
How I Killed My Father
Fathers' Day
Dragon Ball Z: Battle of Gods
The Lawnmower Man
Black God, White Devil
In the Name of the Father
Father of the Bride


#### Matching Score with movie tf_idf value

In [40]:
def get_movie_query(title):
    idx = indices[title]
    tokens = []
    
    for i in tf_idf:
        if i[0] == idx: tokens.append(i[1])
    
    return tokens 

In [49]:
def matching_score_ver1(title):
    tokens = get_movie_query(title)
    
    query_weights = {}
    
    for key in tf_idf:
        if key[1] in tokens:
            try:
                query_weights[key[0]] += tf_idf[key]
            except:
                query_weights[key[0]] = tf_idf[key]
   
    query_weights = sorted(query_weights.items(), key=lambda x: x[1], reverse = True)
    # x[1] is matching score value
    
    print("")
    
    for i in query_weights[1:10]:
        print(smd.iloc[i[0]]['title'])
            

In [50]:
matching_score_ver1("The Dark Knight")


The Dark Knight Rises
Batman: The Dark Knight Returns, Part 2
Henry's Crime
Citizen's Band
Batman
Batman Returns
Batman: Under the Red Hood
Batman Begins
Batman: Mask of the Phantasm


### TF Cosine Similarity Ranking

In [51]:
# hàm tính đố tương đồng giữa 2 vector 
def cosine_sim_calculate(a,b):
    cos_sim = np.dot(a,b)/(np.linalg.norm(a)*np.linalg.norm(b))
    return cos_sim

#### Vectorizing tf-idf

In [52]:
# lập danh sách các vector 
# ma trận D ban đầu có giá trị toàn bằng 0, sau đó chúng ta sẽ duyệt tf_idf và trám các giá trị có tồn tại vào D
D = np.zeros((N, total_vocab_size))
for i in tf_idf:
    try:
        ind = total_vocab.index(i[1]) # vị trí của từ trong vocabolary 
        D[i[0]][ind] = tf_idf[i] 
    except:
            pass

In [53]:
# tạo vector từ query, nếu từ nào không xuất hiện thì tần suất xuất hiện của nó bằng 0
def gen_vector(tokens):
    
    Q = np.zeros(total_vocab_size)
    
    counter = Counter(tokens)
    words_count = len(tokens)
    
    
    for token in np.unique(tokens):
        tf = counter[token]/words_count
        df = doc_freq(token)
        idf = math.log(N/(df + 1))
        
        try:
            ind = total_vocab.index(token)
            Q[ind] = tf*idf
        except:
            pass
    
    return Q

#### Cosine similarity with query string 

In [54]:
#query = "Dark night"

#preprocessed_query = preprocess(query)
#tokens = word_tokenize(str(preprocessed_query))
#array =  gen_vector(tokens)

#for i in array: 
#    print(i)

In [55]:
def cosine_similarity(k, query):
    print("Cosine Similarity")
    preprocessed_query = preprocess(query)
    tokens = word_tokenize(str(preprocessed_query))
    
    print("\n Query:", query)
    print("")
    print(tokens)
    
    d_cosines = []
    
    query_vector = gen_vector(tokens)
    
    for d in D:
        d_cosines.append(cosine_sim_calculate(query_vector, d))
    
    out = np.array(d_cosines).argsort()[-k:][::-1]
    
    print("")
    
    for i in out:
       print(smd.iloc[i]['title']) 
    
    

In [56]:
cosine_similarity(10, "The Dark Knight")

Cosine Similarity

 Query: The Dark Knight

['dark', 'knight']

Last Knights
The Dark Knight Rises
The Hollywood Knights
A Knight's Tale
The Seeker: The Dark Is Rising
Dark Water
The Dark Knight
Dark Water
Black Knight
A Shot in the Dark


In [57]:
Q = cosine_similarity(10, "The Godfather")

Cosine Similarity

 Query: The Godfather

['godfath']

The Godfather: Part III
Tokyo Godfathers
The Godfather
Bright Eyes
The Godfather: Part II
Things to Do in Denver When You're Dead
Jane Austen's Mafia!
Free Enterprise
The Freshman
Cinderfella


#### Cosine similarity with Movie vector 

#### Gen corpus matrix vector using  sklearn.linear_kernel

In [58]:
cosine_sim = linear_kernel(D, D)

In [59]:
cosine_sim[0]

array([0.31099556, 0.00285685, 0.        , ..., 0.        , 0.        ,
       0.00087735])

#### Gen corpus matrix vector from scatch

In [60]:
def cosine_similarity_ver2(k,title):
    idx = indices[title]
    d_cosines = []
    
    for d in D:
        d_cosines.append(cosine_sim_calculate(D[idx],d))
    
    out = np.array(d_cosines).argsort()[::-1]
    
    return out[1:k]

In [64]:
result = cosine_similarity_ver2(11,"The Dark Knight")

In [62]:
f = open('Store_Description_contentBased.pckl', 'wb')
pickle.dump([tf_idf,indices], f)

f.close()

In [63]:
np.save('tf_idf_vector_matrix.npy', D)